## Triangle Spot arbitrage

Fair price:
$$
\text{EUR}JPY_{\text{Ask}} \sim \text{EUR}USD_{\text{Ask}} \times \text{USD}JPY_{\text{Ask}} (Buy)
$$
$$
 EUR\text{JPY}_{\text{Bid}} \sim EUR\text{USD}_{\text{Bid}}\times USD\text{JPY}_{\text{Bid}} (Sell)
$$

> Arbitrage when:
$$
 \text{EUR}jpy_{\text{Ask}} > \text{EUR}usd_{\text{Ask}} \times \text{USD}jpy_{\text{Ask}} (buy-low)
$$(Sell high) $eur\text{JPY}_{\text{Bid}}$
$$
 eur\text{JPY}_{\text{Bid}} < eur\text{USD}_{\text{Bid}}\times usd\text{JPY}_{\text{Bid}} (Sell-high)
$$(Buy low) $\text{EUR}jpy_{\text{Ask}}$


> Long only when:
$$
\text{EUR}jpy_{\text{Ask}} > \text{EUR}usd_{\text{Ask}} \times \text{USD}jpy_{\text{Ask}}
$$(Buy $\text{EUR}usd_{\text{Ask}} \times \text{USD}jpy_{\text{Ask}}$)
$$
eur\text{JPY}_{\text{Bid}} < eur\text{USD}_{\text{Bid}}\times usd\text{JPY}_{\text{Bid}}
$$
(Buy $\text{EUR}jpy_{\text{Ask}}$)

In [ ]:
self.broker.xchg.trading_fee = 0.01 # JPY special pip, 0.0001 normally
tx = 1 - self.broker.xchg.trading_fee # scaled 1 pip for transaction(spread/slippage..etc.)
thres = 0.01 #in Deno(EUR) ## entry at least +0.01 EUR profitable as
## we set base in (JPY)

#fetch pair quotes seperately
quotes={'EURJPY':{
        "ask_price": float(self.broker.get_lowest_ask('EURJPY')),
        "bid_price": float(self.broker.get_highest_bid('EURJPY'))},
    'EURUSD':{
        "ask_price": float(self.broker.get_lowest_ask('EURUSD')),
        "bid_price": float(self.broker.get_highest_bid('EURUSD'))},
    'USDJPY':{
        "ask_price": float(self.broker.get_lowest_ask('USDJPY')),
        "bid_price": float(self.broker.get_highest_bid('USDJPY'))}}
# tradable edge
## buy*buy <sell
implied_lo_ask = (quotes["EURUSD"]["ask_price"]/tx)*(quotes["USDJPY"]["ask_price"]/tx)
long_entry = (implied_lo_ask -thres*1) /tx #in EUR value

## sell*sell >buy
implied_hi_bid = (quotes["EURUSD"]["bid_price"]*tx)*(quotes["USDJPY"]["bid_price"]*tx)
short_entry = (implied_hi_bid -thres*quotes["EURJPY"]["bid_price"]) *tx #in JPY value

## we set base in (JPY) perspective
# be careful: value(Base-> *ask), (Base<- *bid) , action (Base-> *ask/tx), (Base<- *bid*tx)

# buy Deno(EUR) @implied_lo_ask (< ask /tx) |in EUR|> sell Deno(EUR) @hi_bid(= ask+spread) *tx +thres*ask |in JPY|
if long_entry < quotes["EURJPY"]["ask_price"]:

  Qty = 1
  print(f'Buy {quotes.key(2)}: {Qty}@{quotes["USDJPY"]["ask_price"]}\n')
  print(f'Buy {quotes.key(1)}: {Qty}@{quotes["EURUSD"]["ask_price"]}\n')
  print(f'sell {quotes.key(0)}: {Qty}@{quotes["EURJPY"]["bid_price"]}\n') #can try sell @ask_price in 3mins expire if high liquidity
# sell Deno(EUR) @implied_hi_bid (> bid *tx) |in JPY|> buy Deno(EUR) @lo_ask(= bid+spread) /tx +thres |in EUR|
if short_entry > quotes["EURJPY"]["bid_price"]:

  Qty = 1
  print(f'Buy {quotes.key(2)}: {Qty}@{quotes["EURJPY"]["ask_price"]}\n') #can try buy @bid_price in 3mins expire if high liquidity
  print(f'Sell {quotes.key(1)}: {Qty}@{quotes["EURUSD"]["bid_price"]}\n')
  print(f'sell {quotes.key(0)}: {Qty}@{quotes["USDJPY"]["bid_price"]}\n')


In [ ]:
#@title multiprocessing trail
import multiprocessing #consider read data from output of calculating prob density distribution task
import pandas as pd
## buy*buy <sell
def bbs(ask: float)->float:
    long_entry = ((quotes["EURUSD"]["ask_price"]/tx)*(quotes["USDJPY"]["ask_price"]/tx) -thres*1) /tx #in EUR value
    #/return long_entry /
    #generate output table file/weighting/document

## sell*sell >buy
def ssb(bid: float)->float:
    short_entry = ((quotes["EURUSD"]["bid_price"]*tx)*(quotes["USDJPY"]["bid_price"]*tx) -thres*quotes["EURJPY"]["bid_price"]) *tx #in JPY value
    #/return short_entry/
    #generate output table file/weighting/document

if __name__=='__main__':
    #pool = Pool(2) #we don't limit process
    #pool.apply(func=fun1, args=(str, int, list, dict))
    long_entry = multiprocessing.Value('f', quotes["EURUSD"]["bid_price"]) #'f' float 8 no.
    short_entry = multiprocessing.Value('f', quotes["EURUSD"]["ask_price"]) #'f' float 8 no.

    bbs = multiprocessing.Process(target=bbs, arg=(long_entry))
    ssb = multiprocessing.Process(target=ssb, arg=(short_entry))
    bbs.start()
    ssb.start()
    #pool.close()
    bbs.join()
    ssb.join()

    df = pd.read_csv('.csv')
    # buy Deno(EUR) @implied_lo_ask (< ask /tx) |in EUR|> sell Deno(EUR) @hi_bid(= ask+spread) *tx +thres*ask |in JPY|
    if df['long_entry'] < quotes["EURJPY"]["ask_price"] :

      Qty = 1
      print(f'Buy {quotes.key(2)}: {Qty}@{quotes["USDJPY"]["ask_price"]}\n')
      print(f'Buy {quotes.key(1)}: {Qty}@{quotes["EURUSD"]["ask_price"]}\n')
      print(f'sell {quotes.key(0)}: {Qty}@{quotes["EURJPY"]["bid_price"]}\n') #can try sell @ask_price in 3mins expire if high liquidity
    # sell Deno(EUR) @implied_hi_bid (> bid *tx) |in JPY|> buy Deno(EUR) @lo_ask(= bid+spread) /tx +thres |in EUR|
    if df['short_entry'] > quotes["EURJPY"]["bid_price"] :

      Qty = 1
      print(f'Buy {quotes.key(2)}: {Qty}@{quotes["EURJPY"]["ask_price"]}\n') #can try buy @bid_price in 3mins expire if high liquidity
      print(f'Sell {quotes.key(1)}: {Qty}@{quotes["EURUSD"]["bid_price"]}\n')
      print(f'sell {quotes.key(0)}: {Qty}@{quotes["USDJPY"]["bid_price"]}\n')


## Triangle arbitrage with implied USD rate in EURJPY mid-market spot rate

Today is October 27th, 2015. November 3rd is a good business day for all three currencies.

The EURUSD mid-market spot rate is 1.1300 (the price of a EUR in USD) and the USDJPY mid-market spot rate is 120.00 (the price of a USD in JPY). The USD interest rate is 0.25%, the EUR interest rate is 0.50%, and the JPY interest rate is 0.10%.
Use Actural/365 day-count convention, and pick Oct 27 as a common starting point,
$$
\text{EURJPY}^{\text{Nov 3}} = \text{EURJPY}^{\text{Oct 27}}\times \exp\left[(r_{\text{JPY}}-r_{\text{EUR}})\frac{7}{365}\right]\\
\text{USDJPY}^{\text{Nov 2}} = \text{USDJPY}^{\text{Oct 27}}\times \exp\left[(r_{\text{JPY}}-r_{\text{USD}})\frac{6}{365}\right]\\
\text{EURUSD}^{\text{Oct 29}} = \text{EURUSD}^{\text{Oct 27}}\times \exp\left[(r_{\text{USD}}-r_{\text{EUR}})\frac{2}{365}\right]\\
$$
and the triangle relation on Oct 27, (instead of <span style="color:red">triangle</span>, I would rather call this relation <span style="color:red">cyclic</span> as $\text{EURJPY}\times \text{JPYUSD} \times \text{USDEUR} = 1$):
$$
\frac{\text{EURJPY}^{\text{Oct 27}}}{\text{EURUSD}^{\text{Oct 27}} \times \text{USDJPY}^{\text{Oct 27}}} = 1,
$$
we get
$$
\frac{\text{EURJPY}^{\text{Nov 3}} \exp\left[-(r_{\text{JPY}}-r_{\text{EUR}})\frac{7}{365}\right]}{ \text{EURUSD}^{\text{Oct 29}} \exp\left[-(r_{\text{USD}}-r_{\text{EUR}})\frac{2}{365}\right]\times \text{USDJPY}^{\text{Nov 2}}\exp\left[-(r_{\text{JPY}}-r_{\text{USD}})\frac{6}{365}\right]} = 1,
$$
or
\begin{eqnarray}
\text{EURJPY}^{\text{Nov 3}} &=& \text{EURUSD}^{\text{Oct 29}}\times \text{USDJPY}^{\text{Nov 2}}\times \exp\left[-(r_{\text{USD}}-r_{\text{EUR}})\frac{2}{365}-(r_{\text{JPY}}-r_{\text{USD}})\frac{6}{365}+(r_{\text{JPY}}-r_{\text{EUR}})\frac{7}{365}\right]\\
&=& \text{EURUSD}^{\text{Oct 29}}\times \text{USDJPY}^{\text{Nov 2}}\times \exp\left[ r_{\text{USD}}\frac{4}{365} + r_{\text{JPY}}\frac{1}{365} - r_{\text{EUR}}\frac{5}{365}\right]\\
&=& 1.13 \times 120 \times \exp\left[ 0.0025\times\frac{4}{365} + 0.001\times\frac{1}{365} - 0.005\times \frac{5}{365}\right]\\
&\approx& 135.5948.
\end{eqnarray}

Assume zero bid/ask spread in interest rates, Take the bid/ask for EURUSD as 1.1299/1.1301, USDJPY as 119.99/120.01.

The bid/ask for EURJPY implied from the triangle arbitrage:

Using the same formula for bid and ask respectively,
\begin{eqnarray}
\text{EURJPY}^{\text{Nov 3}} &=& \text{EURUSD}^{\text{Oct 29}}\times \text{USDJPY}^{\text{Nov 2}}\times \exp\left[ r_{\text{USD}}\frac{4}{365} + r_{\text{JPY}}\frac{1}{365} - r_{\text{EUR}}\frac{5}{365}\right].
\end{eqnarray}
Thus, with exactly the same discounting factors,
\begin{eqnarray}
\text{EURJPY}^{\text{Nov 3}}_{\text{Ask}} &=& 1.1301 \times 120.01 \times \exp\left[ 0.0025\times\frac{4}{365} + 0.001\times\frac{1}{365} - 0.005\times \frac{5}{365}\right]\\
&\approx& 135.6181.\\
\text{EURJPY}^{\text{Nov 3}}_{\text{Bid}} &=& 1.1299 \times 119.99 \times \exp\left[ 0.0025\times\frac{4}{365} + 0.001\times\frac{1}{365} - 0.005\times \frac{5}{365}\right]\\
&\approx& 135.5715.
\end{eqnarray}

---------------------------
Spot Triangle Arbitrage

$$
\text{EUR}JPY_{\text{Bid}} \sim \text{EUR}USD_{\text{Bid}} \times \text{USD}JPY_{\text{Bid}}
$$

if EURUSD is quoted in USDEUR? Then

$$
EUR\text{JPY}_{\text{Ask}} \sim \frac{USD\text{JPY}_{\text{Ask}}}{\text{USD}EUR_{\text{Bid}}}
$$
In real market, sell in bid price to get reverse rate into value calculate

## Triangle & IRC Factor FX forward shock


PnL calculation: for a forward settling in time $T$, a forward settling in time $T$ which lies between two benchmark settlement dates  $T_1$  and  $T_2$, Derive the notionals $N_1$ and $N_2$ of the benchmark forwards which hedge the portfolio risk assuming triangle shocks to the benchmark non-USD interest rates

 Receiving 1 unit of asset currency, paying $K$ units of denominated currency, is equivalent to being long 1 unit of a zero coupon bond in asset currency and being short $K$ units of a zero coupon bond in denominated currency, the price of which at spot time $t=0$ is
$$
V(T) = S_0 e^{-Q(T)T} - K e^{-R(T)T},
$$
where $S_0$ is the current spot exchange rate. Similarly,
\begin{eqnarray}
V(T_1) = S_0 e^{-Q(T_1)T_1} - K e^{-R(T_1)T_1},\\
V(T_2) = S_0 e^{-Q(T_2)T_2} - K e^{-R(T_2)T_2}.
\end{eqnarray}
Construct a portfolio that's long one unit of $V(T)$, short $N_1$ units of $V(T_1)$ and $N_2$ units of $V(T_2)$:
$$
\Pi = V(T) - N_1 V(T_1) - N_2 V(T_2),
$$
such that the risk to moving rates $Q(T_1)$ and $Q(T_2)$ is hedged:
\begin{eqnarray}
\frac{d \Pi}{d Q(T_1)} = \frac{\partial \Pi}{\partial Q(T_1)} + \frac{\partial \Pi}{\partial Q(T)}\frac{\partial Q(T)}{\partial Q(T_1)} = S_0 \left[N_1 T_1  e^{-Q(T_1)T_1} - T  e^{-Q(T)T} \frac{\partial Q(T)}{\partial Q(T_1)}\right] =0,\\
\frac{d \Pi}{d Q(T_2)} = \frac{\partial \Pi}{\partial Q(T_2)} + \frac{\partial \Pi}{\partial Q(T)}\frac{\partial Q(T)}{\partial Q(T_2)}=  S_0 \left[N_2 T_2  e^{-Q(T_2)T_2} - T e^{-Q(T)T} \frac{\partial Q(T)}{\partial Q(T_1)}\right] =0.
\end{eqnarray}
Assuming a triangle-shock impact to $Q(T)$,
\begin{eqnarray}
\frac{\partial Q(T)}{\partial Q(T_1)} = \frac{T_2 - T}{T_2-T_1}, \text{ shocked at }T_1\text{ and fixed at }T_2\text{, linear interpolate between }T_2\text{ and }T_1\text{ at }T,\\
\frac{\partial Q(T)}{\partial Q(T_2)} = \frac{T - T_1}{T_2-T_1}, \text{ shocked at }T_2\text{ and fixed at }T_1\text{, linear interpolate between }T_2\text{ and }T_1\text{ at }T,
\end{eqnarray}
we get
\begin{eqnarray}
N_1 T_1  e^{-Q(T_1)T_1} - T  e^{-Q(T)T} \left(\frac{T_2 - T}{T_2-T_1}\right) =0 \Rightarrow N_1 = \left(\frac{T_2 - T}{T_2-T_1}\right) \frac{T}{T_1}  e^{-Q(T)(T-T_1)}, \\
N_2 T_2  e^{-Q(T_2)T_2} - T  e^{-Q(T)T} \left(\frac{T - T_1}{T_2-T_1}\right) =0\Rightarrow N_2 = \left(\frac{T - T_1}{T_2-T_1}\right) \frac{T}{T_2}  e^{-Q(T)(T-T_2)}.
\end{eqnarray}


construct a portfolio that's long one unit of $V(T)$, short $N_1$ units of $V(T_1)$ and $N_2$ units of $V(T_2)$:
$$
\Pi = V(T) - N_1 V(T_1) - N_2 V(T_2),
$$

The change in the price of that forward under a shock $dQ(T)$ is then
$$
dV(T) = S_0 \left[ e^{-(Q(T)+dQ(T))T} - e^{-Q(T)T} \right] = S_0e^{-Q(T)T} \left[ e^{-dQ(T)T} - 1\right],
$$
independent of the forward strike $K$ or the interest rate $R$ of the denominated currency. The PnL calculation of the portfolio is thus
$$
dV(T)-N_1 dV(T_1) - N_2 dV(T_2).
$$

-------------------------

For shocks from factors,
\begin{eqnarray}
dQ(T_1) = \sigma_1 e^{-\beta_1 T_1}dz_1 + \sigma_2 e^{-\beta_2 T_1}dz_2,\\
dQ(T_2) = \sigma_1 e^{-\beta_1 T_2}dz_1 + \sigma_2 e^{-\beta_2 T_2}dz_2.
\end{eqnarray}
For $Q(T_1)$ shocked and $Q(T_2)$ fixed,
$$
dQ(T_2) = \sigma_1 e^{-\beta_1 T_2}dz_1 + \sigma_2 e^{-\beta_2 T_2}dz_2 = 0 \Rightarrow \frac{dz_1}{dz_2} = -\frac{\sigma_2 e^{-\beta_2 T_2}}{\sigma_1 e^{-\beta_1 T_2}}.
$$
Substitute back to $dQ(T_1) = \sigma_1 e^{-\beta_1 T_1}dz_1 + \sigma_2 e^{-\beta_2 T_1}dz_2$, we get
\begin{eqnarray}
\sigma_1dz_1 &=& \frac{e^{-\beta_2 T_2}}{e^{-\beta_1 T_1  - \beta_2 T_2 } - e^{-\beta_1 T_2  - \beta_2 T_1  }}dQ(T_1),\\
\sigma_2dz_2 &=& -\frac{e^{-\beta_1 T_2}}{e^{-\beta_1 T_1  - \beta_2 T_2 } - e^{-\beta_1 T_2  - \beta_2 T_1  }}dQ(T_1).
\end{eqnarray}
Thus,
$$
dQ(T) = \sigma_1 e^{-\beta_1 T}dz_1 + \sigma_2 e^{-\beta_2 T}dz_2
=  \frac{e^{-\beta_2 T_2 - \beta_1 T}-e^{-\beta_1 T_2 - \beta_2 T}}{e^{-\beta_1 T_1  - \beta_2 T_2 } - e^{-\beta_1 T_2  - \beta_2 T_1  }}  dQ(T_1) \Rightarrow \frac{\partial Q(T)}{\partial Q(T_1)} = \frac{e^{-\beta_2 T_2 - \beta_1 T}-e^{-\beta_1 T_2 - \beta_2 T}}{e^{-\beta_1 T_1  - \beta_2 T_2 } - e^{-\beta_1 T_2  - \beta_2 T_1  }}.
$$
Similarly, for $Q(T_2)$ shocked and $Q(T_1)$ fixed,
$$
\frac{\partial Q(T)}{\partial Q(T_2)} = \frac{e^{-\beta_1 T_1 - \beta_2 T}-e^{-\beta_2 T_1 - \beta_1 T}}{e^{-\beta_1 T_1  - \beta_2 T_2  } - e^{-\beta_1 T_2  - \beta_2 T_1 } }.
$$

-------------------------

We briefly tabulate the results below,

|  |  Trianglar shock  |   Factor Model shock |
|:------|------|------|
|$\frac{\partial Q(T)}{\partial Q(T_1)}$|$\frac{T_2-\max{(T,T_1)}}{T_2-T_1}$|$\frac{e^{-\beta_2 T_2 - \beta_1 T}-e^{-\beta_1 T_2 - \beta_2 T}}{e^{-\beta_1 T_1  - \beta_2 T_2 } - e^{-\beta_1 T_2  - \beta_2 T_1  }}$|
|$\frac{\partial Q(T)}{\partial Q(T_2)}$|$\frac{\min{(T,T_2)}-T_1}{T_2-T_1}$|$\frac{e^{-\beta_1 T_1 - \beta_2 T}-e^{-\beta_2 T_1 - \beta_1 T}}{e^{-\beta_1 T_1  - \beta_2 T_2  } - e^{-\beta_1 T_2  - \beta_2 T_1 } }$|

Note that, interestingly, the factor model shock is reduced to the triangular shock in the limit $\beta_1,\beta_2\rightarrow 0$, i.e. the limit of weakly decaying shocks:
\begin{eqnarray}
\frac{e^{-\beta_2 T_2 - \beta_1 T}-e^{-\beta_1 T_2 - \beta_2 T}}{e^{-\beta_1 T_1  - \beta_2 T_2 } - e^{-\beta_1 T_2  - \beta_2 T_1  }} \rightarrow \frac{T_2-T}{T_2-T_1},\\
\frac{e^{-\beta_1 T_1 - \beta_2 T}-e^{-\beta_2 T_1 - \beta_1 T}}{e^{-\beta_1 T_1  - \beta_2 T_2  } - e^{-\beta_1 T_2  - \beta_2 T_1 } }\rightarrow \frac{T-T_1}{T_2-T_1}.
\end{eqnarray}

-------------------------

### * Monte Carlo simulation for relative value ( unhedged vs currency interest rate curve shock vs triangular shock)

Let spot = 1, asset currency interest rate curve = $Q(T)$ = flat at 3%, and denominated currency interest rate curve = $R(T)$ = flat at 0%. We assume two benchmark dates, $T_1$ = 0.25y and $T_2$ = 1y; we will use forwards to those settlement dates to hedge the forward rate risk (or equivalently, the risk to moves in the asset currency interest rate) of our portfolio.

In the toy market, we assume that we know the dynamics of the asset currency interest rate:
\begin{eqnarray}
&& dQ = \sigma_1 e^{-\beta_1 T}dz_1 + \sigma_2 e^{-\beta_2 T}dz_2,\\
&& \mathbb{E}[dz_1 dz_2] = \rho dt,
\end{eqnarray}
where $\sigma_1 = 1\%/\sqrt{\text{year}}$, $\sigma_2 = 0.8\%/\sqrt{\text{year}}$, $\beta_1 = 0.5/\text{year}$, $\beta_2 = 0.1/\text{year}$, and $\rho = -0.4$.

>$\sigma\sqrt{\text{T}} = 0.8\%$, $d_2=d_1-\sigma\sqrt{\text{T}}$

The portfolio to hedge has one position: a unit asset-currency notional of a forward contract settling at time $T$. You’ll try this for values of $T$ in [0.1,0.25,0.5,0.75,1,2] to see how performance changes for portfolios with risk to different tenors.

Simulate the portfolio forward a time $dt=0.001\text{y}$. That will result in the asset-currency rates moving according to the factor model described above, which shocks the benchmark rates for tenors $T_1$ and $T_2$, and for the portfolio’s risk tenor T. Construct the PNL distributions for the three hedging approaches. Use unhedged version as benchmark, compare how much more effectively the PNL standard deviation is reduced by hedging according to the true factors vs hedging according to the ad hoc triangle shocks.

In [ ]:
import math
import scipy
import pandas

In [ ]:
class BenchmarkShockSimulator:
    """
        A Monte Carlo simulation of a hedged portfolio:
            1) long an FX forward settling at time T,
            2) short an FX forward settling at time T1,
            3) short an FX forward settling at time T2,
        where asset currency is some non-USD interest rate,
              denominated currency is USD interest rate.
        The rate curve moves according to either
            1) a triangular prescription,
            2) or a parametric 2-factor model.
    """

    def __init__(self, sigma1, sigma2, beta1, beta2, rho, T1, T2, Q):
        """
            sigma1: the volatility of the first shock factor
            sigma2: the volatility of the second shock factor
            beta1:  the decaying parameter of the first shock factor
            beta2:  the decaying parameter of the second shock factor
            rho:    the correlation between the two shock factors
            T1:     the time to settlement of the first benchmark forward tenor
            T2:     the time to settlement of the second benchmark forward tenor
            Q:      the initial (background) asset currency forward curve, assumed to be flat
        """

        if T1 >= T2:
            raise ValueError('Time order T1<T2 is not imposed!')

        self.sigma1 = sigma1
        self.sigma2 = sigma2
        self.beta1 = beta1
        self.beta2 = beta2
        self.rho = rho
        self.T1 = T1
        self.T2 = T2
        self.Q = Q

    def run(self, spot, T, dt, nRuns, hedge, seed, debug=False):
        """
            spot:  spot price of asset
            T:     the settling time of asset-currency notional of a forward contract in portfolio
            dt:    width of a single time step of simulation
            nRuns: number of instantiations of paths
            hedge: 'None' for no hedge;
                   'Triangle' for hedging by triangular shocks;
                   'Factor' for hedging by factor models specified for shock evolution.
        """

        # setup hedging notionals
        if hedge is None:
            N1 = N2 = 0
        elif hedge == 'Triangle':
            if T <= self.T1:
                N1 = 1
                N2 = 0
            elif T >= self.T2:
                N1 = 0
                N2 = 1
            else:
                N1 = (self.T2 - T) / (self.T2 - self.T1)
                N2 = (T - self.T1) / (self.T2 - self.T1)
        elif hedge == 'Factor':
            N1 = (math.exp(-self.beta2 * self.T2 - self.beta1 * T)-math.exp(-self.beta1 * self.T2 - self.beta2 * T))/(math.exp(-self.beta1 * self.T1 - self.beta2 * self.T2)-math.exp(-self.beta1 * self.T2 - self.beta2 * self.T1))
            N2 = (math.exp(-self.beta1 * self.T1 - self.beta2 * T)-math.exp(-self.beta2 * self.T1 - self.beta1 * T))/(math.exp(-self.beta1 * self.T1 - self.beta2 * self.T2)-math.exp(-self.beta1 * self.T2 - self.beta2 * self.T1))
        else:
            raise ValueError('Hedge type must be 1) None 2) Triangle 3) Factor')

        N1 *= (T / self.T1) * math.exp(-self.Q * (T - self.T1))
        N2 *= (T / self.T2) * math.exp(-self.Q * (T - self.T2))

        # generate instances of shocks
        scipy.random.seed(seed)
        sqrtdt = math.sqrt(dt)
        # independent Brownian shocks
        dw1 = scipy.random.normal(0, sqrtdt, nRuns)
        dw2 = scipy.random.normal(0, sqrtdt, nRuns)
        # correlated Brownian shocks by rho
        dz1 = dw1
        dz2 = self.rho * dw1 + math.sqrt(1 - self.rho * self.rho) * dw2

        dQT = self.sigma1 * math.exp(-self.beta1 * T)  * dz1 + self.sigma2 * math.exp(-self.beta2 * T)  * dz2
        dQ1 = self.sigma1 * math.exp(-self.beta1 * self.T1) * dz1 + self.sigma2 * math.exp(-self.beta2 * self.T1) * dz2
        dQ2 = self.sigma1 * math.exp(-self.beta1 * self.T2) * dz1 + self.sigma2 * math.exp(-self.beta2 * self.T2) * dz2

        # calculate the portfolio PnLs under the rate shocks
        # Note: in the second exp function, needs to use scipy version to vectorize.
        pnls = spot * math.exp(-self.Q * T) * (scipy.exp(-dQT * T) - 1)
        pnls -= N1 * spot * math.exp(-self.Q * self.T1) * (scipy.exp(-dQ1 * T1) - 1)
        pnls -= N2 * spot * math.exp(-self.Q * self.T2) * (scipy.exp(-dQ2 * T2) - 1)

        return (pnls.mean(), pnls.std())


In [ ]:
# Static parameters
sigma1 = 0.01
sigma2 = 0.008
beta1 = 0.5
beta2 = 0.1
rho = -0.4
T1 = 0.25
T2 = 1
Q = 0.03

In [ ]:
# Create simulator
sim = BenchmarkShockSimulator(sigma1, sigma2, beta1, beta2, rho, T1, T2, Q)

In [ ]:
# Simulation parameters
spot = 1
dt = 1e-3
nRuns = 100000
seed = 1
Ts = [0.1, 0.25, 0.5, 0.75, 1, 2]
hedgeTypes = [None,'Triangle','Factor']

df = pandas.DataFrame()
df['T'] = Ts

In [ ]:
# Run simulations
for hedgeType in hedgeTypes:
    # choose a nice name for each hedge type
    if hedgeType is None:
        hedgeTypeName = 'No Hedge'
    else:
        hedgeTypeName = hedgeType

    # run experiments for different settling times
    stds = []
    for T in Ts:
        mean, std = sim.run(spot, T, dt, nRuns, hedgeType, seed)
        stds.append(std*1e3)

    df[hedgeTypeName] = stds

In [ ]:
# display the standard deviations in units of 10^-3.
df['Triangle efficiency'] = 1-df['Triangle']/df['No Hedge']
df['Factor efficiency'] = 1-df['Factor']/df['No Hedge']
df.round(6)

,T,No Hedge,Triangle,Factor,Triangle efficiency,Factor efficiency
0,0.10,0.030238,0.002042,0.000000,0.932486,0.999988
1,0.25,0.071404,0.000000,0.000000,1.000000,1.000000
2,0.50,0.130707,0.001792,0.000001,0.986290,0.999990
3,0.75,0.180956,0.002558,0.000002,0.985864,0.999990
4,1.00,0.224536,0.000000,0.000000,1.000000,1.000000
5,2.00,0.361658,0.130804,0.000033,0.638321,0.999909


 The declining in the curve backend is beccause the assumption of forward curve moving in parallel in the backend (from 1y outward) is unrealistic. In fact, the backend of the forward curve responds to a less extent to the shocks than the front end does, the 2y tenor will move less than the 1y tenor. As a consequence, the triangular shock model over-estimates the hedging notional on 1y tenor.

The factor model hedging essentially offsets all movements in $Q(T)$ (>0.9999). This is expected because the shocks are hedged in exactly the same way they are generated, according to the 2-factor stochastic differential equations.

### Principal component analysis and factor models are two different methods for reducing dimensionality in a high-dimensional system:



| Features |  Principle Component Analysis (PCA)   |   Factor Models |
|:------|------|------|
| Parameters     | Non-Parametric |  Parametric, parameterized by $\sigma_1, \sigma_2, \beta_1, \beta_2, \rho$, etc.    |
|General idea|Discovering the main factors driving the moves in the interest rate curve|Fitting historical behavior to a prescribed parametric form of factors|
| Methodology | Calculating the historical covariances of daily changes in interest rates of different tenors, then orthogonalizing the covariance matrix|Solving stochastic differential equation driving interest rates of different tenors, the computing the covariance matrix analytically in terms of the parameters|
| Shocks to curve | Eigenvectors of historical covariance matrix of daily changes in different tenors, representing othogonal shocks to curve | Shocked by a small number of Brownian motion factors in a stochastic differential equation driving different tenors|
|How to reduce dimensionality|Picking off the largest few eigenvalues and approximating all moves in the curves as being linear combinations of the respective eigenvectors|A few parametric factor in the model specification|
|Shape of shocks|Non-parametric output of the calculation|Determined by the model parameters, for example, exponential shocks|
|Strength|Flexibile in terms of shape of the shocks|Robust in evaluating the shocks and easier for traders to understand|
|Weakness|Less robust (historical data artifacts can lead to weird shapes)|Less flexibile in capturing shocks|